In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/BearingProject/scripts')


In [3]:
from SDA_data_utils import (
    load_cwru_signals, load_paderborn_signals,
    segment_signals, maybe_resample_list
)

from SDA_model_utils import build_cnn1d


In [ ]:
!python /content/drive/MyDrive/BearingProject/scripts/SDA_data_utils.py

In [14]:
# -*- coding: utf-8 -*-

import os
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers, losses

from SDA_data_utils import (
    load_cwru_signals, load_paderborn_signals,
    segment_signals, maybe_resample_list
)

from SDA_model_utils import build_cnn1d

# ==========================================
# CONFIG
# ==========================================
SOURCE_KEY = "12K3HP"
TARGET_COND = "Condition2"

KFOLD = 5
PRETRAIN_EPOCHS = 200
BATCH_SIZE = 64
PATIENCE = 50

USE_TARGET_FRACTION = 1.0
SEED = 42
VERBOSE = True


INTERVAL_LENGTH = 320
SAMPLES_PER_BLOCK = 1600

# Explainability settings
NUM_EXPLAIN_PER_CLASS = 5

OUTPUT_ROOT = "12K3HP_COND1"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

np.random.seed(SEED)
tf.random.set_seed(SEED)

# ==========================================
# UTILITY FUNCTIONS
# ==========================================
def plot_confusion_matrix(y_true, y_pred, title, save_path):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(title)
    plt.savefig(save_path)
    plt.close()


def plot_training_history(history_log, fold_dir, fold):
    epochs = [h["epoch"] for h in history_log]
    train_loss = [h["train_loss"] for h in history_log]
    val_acc = [h["val_acc"] for h in history_log]

    plt.figure()
    plt.plot(epochs, train_loss, label="Train Loss")
    plt.plot(epochs, val_acc, label="Val Acc")
    plt.xlabel("Epoch")
    plt.ylabel("Value")
    plt.title(f"Fold {fold} Training History")
    plt.legend()
    plt.savefig(os.path.join(fold_dir, f"Fold{fold}_History.png"))
    plt.close()


In [15]:
def get_last_conv_layer(model):
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv1D):
            return layer.name
    raise ValueError("Model has no Conv1D layers for Grad-CAM.")


# ==========================================
# EXPLAINABILITY — GRAD-CAM
# ==========================================
def compute_gradcam(model, sample, pred_index, last_conv_name="conv1d_3"):
    """Compute 1D Grad-CAM """
    last_conv_layer = model.get_layer(last_conv_name)

    # Create a gradient model
    grad_model = tf.keras.models.Model(
        inputs=model.input,
        outputs=[last_conv_layer.output, model.output]
    )

    # Convert sample to tensor
    sample_tensor = tf.cast(sample, tf.float32)

    with tf.GradientTape() as tape:
        # Get conv output and predictions in one forward pass
        conv_output, preds = grad_model(sample_tensor, training=False)

        # Get the score for the predicted class
        class_channel = preds[:, pred_index]

    # Compute gradients of the class score with respect to conv output
    grads = tape.gradient(class_channel, conv_output)

    # Global average pooling on gradients
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))

    # Get theconv output as numpy
    conv_output = conv_output[0].numpy()  # (T, C)
    pooled_grads = pooled_grads.numpy()   # (C,)

    # Weight each channel by the corresponding gradient
    for i in range(conv_output.shape[-1]):
        conv_output[:, i] *= pooled_grads[i]

    # Create heatmap by averaging across channels
    heatmap = np.mean(conv_output, axis=-1)

    # ReLU - only keep positive contributions
    heatmap = np.maximum(heatmap, 0)

    # Normalize to [0, 1]
    if np.max(heatmap) > 0:
        heatmap = heatmap / np.max(heatmap)

    return heatmap


def plot_gradcam(sample_1d, heatmap, true_label, pred_label,
                 class_names, save_path):
    T = len(sample_1d)
    time = np.arange(T)

    heatmap_resized = np.interp(
        np.linspace(0, len(heatmap) - 1, T),
        np.arange(len(heatmap)),
        heatmap
    )

    fig, axs = plt.subplots(3,1,figsize=(14,9))

    # Raw signal
    axs[0].plot(time, sample_1d, linewidth=0.7)
    axs[0].set_title(f"Signal — True: {class_names[true_label]}, Pred: {class_names[pred_label]}")
    axs[0].grid(True, alpha=0.3)

    # Heatmap
    axs[1].imshow(heatmap_resized.reshape(1,-1), cmap="jet",
                  aspect="auto", extent=[0,T,0,1])
    axs[1].set_title("Grad-CAM Heatmap (Red = high importance)")
    axs[1].set_yticks([])

    # Overlay
    axs[2].plot(time, sample_1d, alpha=0.7)
    axs[2].imshow(heatmap_resized.reshape(1,-1), cmap="jet",
                  alpha=0.4, aspect="auto",
                  extent=[0,T, sample_1d.min(), sample_1d.max()])
    axs[2].set_title("Overlay: Signal + Grad-CAM")
    axs[2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()


# ==========================================
# EXPLAINABILITY — IMPROVED LRP (SmoothGrad + IG)
# ==========================================
class ImprovedLRP:
    def __init__(self, model):
        self.model = model

    def compute_relevance(self, sample, pred_index,
                          num_samples=30, noise_level=0.15):
        baseline = np.zeros_like(sample)
        T = sample.shape[1]

        signal_range = np.max(sample) - np.min(sample)
        stdev = noise_level * signal_range if signal_range > 0 else 0.01

        total_gradients = np.zeros_like(sample)

        for _ in range(num_samples):
            noise = np.random.normal(0, stdev, sample.shape)
            noisy_input = sample + noise

            alphas = np.linspace(0, 1, 20)
            path_grads = []

            for a in alphas:
                x = baseline + a * (noisy_input - baseline)
                with tf.GradientTape() as tape:
                    x_tf = tf.Variable(x, dtype=tf.float32)
                    preds = self.model(x_tf)
                    target = preds[:, pred_index]

                grads = tape.gradient(target, x_tf)
                if grads is not None:
                    path_grads.append(grads.numpy())

            if len(path_grads) > 0:
                avg_grad = np.mean(path_grads, axis=0)
                total_gradients += avg_grad

        smooth_grads = total_gradients / num_samples

        relevance = (sample - baseline) * smooth_grads
        relevance = relevance[0,:,0]  # flatten to 1D

        # Gaussian smoothing
        from scipy.ndimage import gaussian_filter1d
        relevance_smooth = gaussian_filter1d(relevance, sigma=2.5)

        return relevance_smooth


def plot_lrp(sample_1d, relevance, true_label, pred_label,
             class_names, save_path):
    """Plot sample, LRP relevance, absolute relevance, and colored overlay."""
    T = len(sample_1d)
    time = np.arange(T)

    abs_rel = np.abs(relevance)
    top_idx = np.argsort(abs_rel)[-5:]  # top 5 points

    fig, axs = plt.subplots(4,1,figsize=(14,11))

    # Raw signal
    axs[0].plot(time, sample_1d)
    axs[0].set_title(f"Signal — True: {class_names[true_label]}, Pred: {class_names[pred_label]}")
    axs[0].grid(True, alpha=0.3)

    # Smoothed relevance
    axs[1].plot(time, relevance, color='red')
    axs[1].set_title("LRP Relevance (Smoothed)")
    axs[1].axhline(0, linestyle="--", alpha=0.4)
    axs[1].grid(True, alpha=0.3)

    # Absolute relevance
    axs[2].plot(time, abs_rel, color='darkred')
    axs[2].fill_between(time, abs_rel, alpha=0.4)
    axs[2].scatter(time[top_idx], abs_rel[top_idx],
                   c='yellow', edgecolors='black', s=80, label="Top Points")
    axs[2].legend()
    axs[2].set_title("|Relevance| Strongest Regions")
    axs[2].grid(True, alpha=0.3)

    # Overlay scatter
    rel_norm = (relevance - relevance.min()) / (relevance.max() - relevance.min() + 1e-9)
    axs[3].scatter(time, sample_1d, c=rel_norm, cmap="RdYlBu_r", s=4)
    axs[3].set_title("Signal with LRP Coloring")
    axs[3].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()


In [16]:
# ==========================================
# GENERATE EXPLAINABILITY FOR ONE FOLD
# ==========================================
def generate_explainability_for_fold(model, X, y, class_names, save_dir, prefix):
    os.makedirs(save_dir, exist_ok=True)
    gradcam_dir = os.path.join(save_dir, "GradCAM")
    lrp_dir = os.path.join(save_dir, "LRP")
    os.makedirs(gradcam_dir, exist_ok=True)
    os.makedirs(lrp_dir, exist_ok=True)

        # Debug: Check label values
    print(f"Unique labels in y: {np.unique(y)}")
    print(f"Class names length: {len(class_names)}")
    print(f"Class names: {class_names}")

    preds = np.argmax(model.predict(X, verbose=0), axis=1)

    # Debug: Check prediction values
    print(f"Unique predictions: {np.unique(preds)}")

    lrp = ImprovedLRP(model)

    for cls in np.unique(y):
        idx_cls = np.where(y == cls)[0]

        # pick 5 samples
        chosen = idx_cls[:NUM_EXPLAIN_PER_CLASS]

        for i, idx in enumerate(chosen):
            sample = X[idx:idx+1]
            sample_1d = sample[0,:,0]
            true_lbl = int(y[idx])  # Ensure it's an int
            pred_lbl = int(preds[idx])  # Ensure it's an int

            # -----------------------
            # GRAD-CAM
            # -----------------------
            last_conv_name = get_last_conv_layer(model)
            heatmap = compute_gradcam(model, sample, pred_lbl, last_conv_name)
            save_gc = os.path.join(
                gradcam_dir,
                f"{prefix}_class{cls}_sample{i+1}_pred{pred_lbl}.png"
            )
            plot_gradcam(sample_1d, heatmap, true_lbl, pred_lbl,
                         class_names, save_gc)

            # -----------------------
            # LRP
            # -----------------------
            relevance = lrp.compute_relevance(sample, pred_lbl)
            save_lrp = os.path.join(
                lrp_dir,
                f"{prefix}_class{cls}_sample{i+1}_pred{pred_lbl}.png"
            )
            plot_lrp(sample_1d, relevance, true_lbl, pred_lbl,
                     class_names, save_lrp)


# ==========================================
# MAIN TRAINING WITH 5-FOLD CV
# ==========================================
def train_no_da_kfold(src_signals, src_labels, tgt_signals, tgt_labels,
                      src_rates, tgt_rates):
    # Output folder
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_dir = os.path.join(OUTPUT_ROOT, f"NoDA_{timestamp}")
    os.makedirs(base_dir, exist_ok=True)

    # Resample to lowest source rate
    target_fs = min(src_rates)
    src_res = maybe_resample_list(src_signals, src_rates, target_fs)
    tgt_res = maybe_resample_list(tgt_signals, tgt_rates, target_fs)

    X_src, y_src = segment_signals(src_res, src_labels,
                                   INTERVAL_LENGTH, SAMPLES_PER_BLOCK)
    X_tgt, y_tgt = segment_signals(tgt_res, tgt_labels,
                                   INTERVAL_LENGTH, SAMPLES_PER_BLOCK)

    print("Source samples:", len(X_src))
    print("Target samples:", len(X_tgt))

    # 5-fold split on SOURCE only
    kfold = KFold(n_splits=KFOLD, shuffle=True, random_state=SEED)

    fold_results_src = []
    fold_results_tgt = []

    class_names = ["Healthy", "OR_Fault", "IR_Fault"]

    X_t_test = X_tgt  # Use all target data
    y_t_test = y_tgt
    print(f"Target test samples: {len(X_t_test)}")
    fold = 1
    for train_idx, val_idx in kfold.split(X_src, y_src):
        print(f"\n========== FOLD {fold}/{KFOLD} ==========")
        fold_dir = os.path.join(base_dir, f"Fold_{fold}")
        os.makedirs(fold_dir, exist_ok=True)

        # Split source into train/val
        Xs_train, Xs_val = X_src[train_idx], X_src[val_idx]
        ys_train, ys_val = y_src[train_idx], y_src[val_idx]

        # Build CNN
        num_classes = len(np.unique(y_src))
        model = build_cnn1d(X_src.shape[1:], num_classes)
        opt = optimizers.Adam(1e-4)
        loss_fn = losses.SparseCategoricalCrossentropy()

        best_val = 0
        epochs_no_improve = 0
        history_log = []
        best_path = os.path.join(fold_dir, "best_model.h5")

        # -------------------------------
        # TRAINING LOOP (NO DA)
        # -------------------------------
        for ep in range(PRETRAIN_EPOCHS):

            # Shuffle source
            idx = np.random.permutation(len(Xs_train))
            Xs_train_sh = Xs_train[idx]
            ys_train_sh = ys_train[idx]

            num_batches = max(len(Xs_train_sh) // BATCH_SIZE, 1)
            train_loss_accum = 0

            for b in range(num_batches):
                start = b * BATCH_SIZE
                end = start + BATCH_SIZE
                xb = Xs_train_sh[start:end]
                yb = ys_train_sh[start:end]

                with tf.GradientTape() as tape:
                    logits = model(xb, training=True)
                    loss = loss_fn(yb, logits)

                grads = tape.gradient(loss, model.trainable_variables)
                opt.apply_gradients(zip(grads, model.trainable_variables))
                train_loss_accum += float(loss.numpy())

            # Validation
            val_logits = model.predict(Xs_val, verbose=0)
            val_preds = np.argmax(val_logits, axis=1)
            val_acc = accuracy_score(ys_val, val_preds)
            val_loss = np.mean(
                losses.sparse_categorical_crossentropy(ys_val, val_logits).numpy()
            )

            history_log.append({
                "epoch": ep + 1,
                "train_loss": train_loss_accum / num_batches,
                "val_acc": val_acc,
                "val_loss": val_loss
            })

            print(f"Epoch {ep+1}: TrainLoss={train_loss_accum/num_batches:.4f}  ValAcc={val_acc:.4f}")

            # Early stopping
            if val_acc > best_val:
                best_val = val_acc
                model.save(best_path)
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= PATIENCE:
                print("Early stopping.")
                break

        # Load best model
        model = tf.keras.models.load_model(best_path)

        model.compile(
            optimizer=optimizers.Adam(1e-4),
            loss=losses.SparseCategoricalCrossentropy(),
            metrics=['accuracy']
        )

        # Warm up the model with a prediction
        _ = model.predict(np.zeros((1, X_src.shape[1], 1), dtype=np.float32), verbose=0)


        # -------------------------------
        # EVALUATE — SOURCE VAL
        # -------------------------------
        y_pred_val = np.argmax(model.predict(Xs_val, verbose=0), axis=1)
        fold_results_src.append([
            accuracy_score(ys_val, y_pred_val),
            precision_score(ys_val, y_pred_val, average="macro", zero_division=0),
            recall_score(ys_val, y_pred_val, average="macro", zero_division=0),
            f1_score(ys_val, y_pred_val, average="macro", zero_division=0)
        ])
        plot_confusion_matrix(
            ys_val, y_pred_val,
            f"Fold {fold} — Source ConfMat",
            os.path.join(fold_dir, "ConfMat_Source.png")
        )
        plot_training_history(history_log, fold_dir, fold)

        # -------------------------------
        # EVALUATE — TARGET TEST
        # -------------------------------
        y_pred_tgt = np.argmax(model.predict(X_t_test, verbose=0), axis=1)
        fold_results_tgt.append([
            accuracy_score(y_t_test, y_pred_tgt),
            precision_score(y_t_test, y_pred_tgt, average="macro", zero_division=0),
            recall_score(y_t_test, y_pred_tgt, average="macro", zero_division=0),
            f1_score(y_t_test, y_pred_tgt, average="macro", zero_division=0)
        ])
        plot_confusion_matrix(
            y_t_test, y_pred_tgt,
            f"Fold {fold} — Target ConfMat",
            os.path.join(fold_dir, "ConfMat_Target.png")
        )

        # -------------------------------
        # TSNE (only fold 5 like supervisor)
        # -------------------------------
        if fold == KFOLD:
            feat_model = models.Model(inputs=model.input,
                                      outputs=model.layers[-3].output)
            src_feats = feat_model.predict(Xs_val)
            tgt_feats = feat_model.predict(X_t_test)

            tsne = TSNE(n_components=2, random_state=SEED)
            final_feats = tsne.fit_transform(np.vstack([src_feats, tgt_feats]))
            labels = np.array([0]*len(src_feats) + [1]*len(tgt_feats))

            plt.figure()
            plt.scatter(final_feats[labels==0,0], final_feats[labels==0,1],
                        label="Source", alpha=0.6)
            plt.scatter(final_feats[labels==1,0], final_feats[labels==1,1],
                        label="Target", alpha=0.6)
            plt.legend()
            plt.title("Fold 5 — t-SNE Source vs Target")
            plt.savefig(os.path.join(fold_dir, "TSNE.png"))
            plt.close()

        # -------------------------------
        # EXPLAINABILITY (only Fold 5)
        # -------------------------------
        if fold == KFOLD:
            print("\nGenerating explainability for Fold 5...")
            exp_dir_src = os.path.join(fold_dir, "Explainability_Source")
            exp_dir_tgt = os.path.join(fold_dir, "Explainability_Target")

            generate_explainability_for_fold(
                model, Xs_val, ys_val, class_names,
                exp_dir_src,
                prefix="Source"
            )
            generate_explainability_for_fold(
                model, X_t_test, y_t_test, class_names,
                exp_dir_tgt,
                prefix="Target"
            )

        fold += 1
   # ==========================================
    # CREATE DATAFRAMES FIRST
    # ==========================================
    df_src = pd.DataFrame(fold_results_src,
                          columns=["Acc","Prec","Rec","F1"])
    df_tgt = pd.DataFrame(fold_results_tgt,
                          columns=["Acc","Prec","Rec","F1"])
    df_src["Type"] = "Source"
    df_tgt["Type"] = "Target"
    df_src["Fold"] = [f"Fold_{i+1}" for i in range(len(df_src))]
    df_tgt["Fold"] = [f"Fold_{i+1}" for i in range(len(df_tgt))]

    df_final = pd.concat([df_src, df_tgt], ignore_index=True)

    # ==========================================
    # PRINT SUMMARY TABLE
    # ==========================================
    print("\n" + "="*80)
    print("FINAL CROSS-DOMAIN RESULTS SUMMARY (5-FOLD CROSS-VALIDATION)")
    print("="*80)

    # Calculate means and stds
    src_mean = df_src[["Acc","Prec","Rec","F1"]].mean()
    src_std = df_src[["Acc","Prec","Rec","F1"]].std()
    tgt_mean = df_tgt[["Acc","Prec","Rec","F1"]].mean()
    tgt_std = df_tgt[["Acc","Prec","Rec","F1"]].std()

    # Print Source Results
    print("\n📊 SOURCE DOMAIN - CWRU (Validation Accuracy):")
    print("   (Model trained on CWRU train set, evaluated on CWRU validation set)")
    print("-" * 80)
    print(f"{'Fold':<10} {'Accuracy':<15} {'Precision':<15} {'Recall':<15} {'F1-Score':<15}")
    print("-" * 80)
    for i, row in df_src.iterrows():
        print(f"{row['Fold']:<10} {row['Acc']*100:>6.2f}%{'':<7} {row['Prec']*100:>6.2f}%{'':<7} "
              f"{row['Rec']*100:>6.2f}%{'':<7} {row['F1']*100:>6.2f}%")
    print("-" * 80)
    print(f"{'MEAN':<10} {src_mean['Acc']*100:>6.2f}%{'':<7} {src_mean['Prec']*100:>6.2f}%{'':<7} "
          f"{src_mean['Rec']*100:>6.2f}%{'':<7} {src_mean['F1']*100:>6.2f}%")
    print(f"{'STD':<10} {src_std['Acc']*100:>6.2f}%{'':<7} {src_std['Prec']*100:>6.2f}%{'':<7} "
          f"{src_std['Rec']*100:>6.2f}%{'':<7} {src_std['F1']*100:>6.2f}%")

    # Print Target Results
    print("\n📊 TARGET DOMAIN - Paderborn (Test Accuracy - CROSS-DOMAIN):")
    print("   (Model trained on CWRU, evaluated on Paderborn test set)")
    print("-" * 80)
    print(f"{'Fold':<10} {'Accuracy':<15} {'Precision':<15} {'Recall':<15} {'F1-Score':<15}")
    print("-" * 80)
    for i, row in df_tgt.iterrows():
        print(f"{row['Fold']:<10} {row['Acc']*100:>6.2f}%{'':<7} {row['Prec']*100:>6.2f}%{'':<7} "
              f"{row['Rec']*100:>6.2f}%{'':<7} {row['F1']*100:>6.2f}%")
    print("-" * 80)
    print(f"{'MEAN':<10} {tgt_mean['Acc']*100:>6.2f}%{'':<7} {tgt_mean['Prec']*100:>6.2f}%{'':<7} "
          f"{tgt_mean['Rec']*100:>6.2f}%{'':<7} {tgt_mean['F1']*100:>6.2f}%")
    print(f"{'STD':<10} {tgt_std['Acc']*100:>6.2f}%{'':<7} {tgt_std['Prec']*100:>6.2f}%{'':<7} "
          f"{tgt_std['Rec']*100:>6.2f}%{'':<7} {tgt_std['F1']*100:>6.2f}%")

    # Domain Gap Analysis
    print("\n📉 CROSS-DOMAIN TRANSFER PERFORMANCE:")
    print("-" * 80)
    acc_gap = (src_mean['Acc'] - tgt_mean['Acc']) * 100
    f1_gap = (src_mean['F1'] - tgt_mean['F1']) * 100
    transfer_rate = (tgt_mean['Acc'] / src_mean['Acc']) * 100 if src_mean['Acc'] > 0 else 0
    print(f"Source Domain (CWRU) Accuracy:     {src_mean['Acc']*100:.2f}% ± {src_std['Acc']*100:.2f}%")
    print(f"Target Domain (Paderborn) Accuracy: {tgt_mean['Acc']*100:.2f}% ± {tgt_std['Acc']*100:.2f}%")
    print(f"Accuracy Drop:                      {acc_gap:.2f}%")
    print(f"Transfer Success Rate:              {transfer_rate:.2f}%")
    print(f"F1-Score Drop:                      {f1_gap:.2f}%")
    print("="*80)

    print(f"\n💡 Interpretation:")
    print(f"   - High source accuracy ({src_mean['Acc']*100:.2f}%) shows model learned CWRU data well")
    print(f"   - Lower target accuracy ({tgt_mean['Acc']*100:.2f}%) shows domain shift challenge")
    print(f"   - The {acc_gap:.2f}% drop indicates the need for domain adaptation methods")
    print("="*80)

    # ==========================================
    # SAVE EXCEL
    # ==========================================
    df_final.to_excel(os.path.join(base_dir, "Summary.xlsx"), index=False)
    print(f"\n Saved summary → {os.path.join(base_dir, 'Summary.xlsx')}")


# ==========================================
# MAIN
# ==========================================
if __name__ == "__main__":
    print("Loading datasets...")

    src_signals, src_labels, src_rates = load_cwru_signals(SOURCE_KEY)
    tgt_signals, tgt_labels, tgt_rates = load_paderborn_signals(TARGET_COND)

    print("Running 5-Fold Cross-Domain Training (NO DA)...")
    train_no_da_kfold(src_signals, src_labels, tgt_signals, tgt_labels,
                      src_rates, tgt_rates)


Loading datasets...
Running 5-Fold Cross-Domain Training (NO DA)...
Source samples: 2264
Target samples: 432
Target test samples: 432

========== FOLD 1/5 ==========


Epoch 1: TrainLoss=0.2299  ValAcc=0.6556
Epoch 2: TrainLoss=0.0267  ValAcc=0.6556


Epoch 3: TrainLoss=0.0179  ValAcc=0.7285
Epoch 4: TrainLoss=0.0126  ValAcc=0.1656
Epoch 5: TrainLoss=0.0106  ValAcc=0.1656
Epoch 6: TrainLoss=0.0089  ValAcc=0.1656
Epoch 7: TrainLoss=0.0065  ValAcc=0.1656
Epoch 8: TrainLoss=0.0060  ValAcc=0.1656
Epoch 9: TrainLoss=0.0053  ValAcc=0.1656
Epoch 10: TrainLoss=0.0062  ValAcc=0.1656
Epoch 11: TrainLoss=0.0046  ValAcc=0.1656
Epoch 12: TrainLoss=0.0033  ValAcc=0.1656
Epoch 13: TrainLoss=0.0031  ValAcc=0.1656
Epoch 14: TrainLoss=0.0035  ValAcc=0.1656
Epoch 15: TrainLoss=0.0026  ValAcc=0.1656
Epoch 16: TrainLoss=0.0029  ValAcc=0.1656
Epoch 17: TrainLoss=0.0030  ValAcc=0.1656
Epoch 18: TrainLoss=0.0022  ValAcc=0.1656
Epoch 19: TrainLoss=0.0020  ValAcc=0.1656


Epoch 20: TrainLoss=0.0017  ValAcc=0.8212
Epoch 21: TrainLoss=0.0016  ValAcc=0.8212


Epoch 22: TrainLoss=0.0021  ValAcc=0.9338


...

Epoch 1: TrainLoss=0.5695  ValAcc=0.8252
Epoch 2: TrainLoss=0.0481  ValAcc=0.6571
Epoch 3: TrainLoss=0.0267  ValAcc=0.6571
Epoch 4: TrainLoss=0.0142  ValAcc=0.6571
Epoch 5: TrainLoss=0.0122  ValAcc=0.6571
Epoch 6: TrainLoss=0.0128  ValAcc=0.6571
Epoch 7: TrainLoss=0.0094  ValAcc=0.6571
Epoch 8: TrainLoss=0.0097  ValAcc=0.6571
Epoch 9: TrainLoss=0.0076  ValAcc=0.7876
Epoch 10: TrainLoss=0.0070  ValAcc=0.8230


Epoch 11: TrainLoss=0.0069  ValAcc=0.9867


Epoch 12: TrainLoss=0.0065  ValAcc=0.9978
Epoch 13: TrainLoss=0.0046  ValAcc=0.9912
Epoch 14: TrainLoss=0.0057  ValAcc=0.9668
Epoch 15: TrainLoss=0.0045  ValAcc=0.9513
Epoch 16: TrainLoss=0.0042  ValAcc=0.9646
Epoch 17: TrainLoss=0.0043  ValAcc=0.9757
Epoch 18: TrainLoss=0.0027  ValAcc=0.9823
Epoch 19: TrainLoss=0.0029  ValAcc=0.9934


Epoch 20: TrainLoss=0.0033  ValAcc=1.0000
Epoch 21: TrainLoss=0.0035  ValAcc=1.0000
Epoch 22: TrainLoss=0.0029  ValAcc=1.0000
Epoch 23: TrainLoss=0.0026  ValAcc=1.0000
Epoch 24: TrainLoss=0.0029  ValAcc=1.0000
Epoch 25: TrainLoss=0.0020  ValAcc=1.0000
Epoch 26: TrainLoss=0.0029  ValAcc=1.0000
Epoch 27: TrainLoss=0.0021  ValAcc=1.0000
Epoch 28: TrainLoss=0.0028  ValAcc=1.0000
Epoch 29: TrainLoss=0.0023  ValAcc=1.0000
Epoch 30: TrainLoss=0.0020  ValAcc=1.0000
Epoch 31: TrainLoss=0.0018  ValAcc=1.0000
Epoch 32: TrainLoss=0.0020  ValAcc=1.0000
Epoch 33: TrainLoss=0.0022  ValAcc=1.0000
Epoch 34: TrainLoss=0.0014  ValAcc=1.0000
Epoch 35: TrainLoss=0.0013  ValAcc=1.0000
Epoch 36: TrainLoss=0.0019  ValAcc=1.0000
Epoch 37: TrainLoss=0.0014  ValAcc=1.0000
Epoch 38: TrainLoss=0.0014  ValAcc=1.0000
Epoch 39: TrainLoss=0.0013  ValAcc=1.0000
Epoch 40: TrainLoss=0.0012  ValAcc=1.0000
Epoch 41: TrainLoss=0.0015  ValAcc=1.0000
Epoch 42: TrainLoss=0.0012  ValAcc=1.0000
Epoch 43: TrainLoss=0.0013  ValAcc

Epoch 70: TrainLoss=0.0007  ValAcc=1.0000
Early stopping.
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step

Generating explainability for Fold 5...
Unique labels in y: [0 1 2]
Class names length: 3
Class names: ['Healthy', 'OR_Fault', 'IR_Fault']
Unique predictions: [0 1 2]
Unique labels in y: [0 1 2]
Class names length: 3
Class names: ['Healthy', 'OR_Fault', 'IR_Fault']
Unique predictions: [1 2]

FINAL CROSS-DOMAIN RESULTS SUMMARY (5-FOLD CROSS-VALIDATION)

📊 SOURCE DOMAIN - CWRU (Validation Accuracy):
   (Model trained on CWRU train set, evaluated on CWRU validation set)
--------------------------------------------------------------------------------
Fold       Accuracy        Precision       Recall          F1-Score       
--------------------------------------------------------------------------------
Fold_1     100.00%        100.00%        100.00%        100.00%
Fold_2     100.00%        100.00%        100.00%        100.00%
Fold_3     100.00%      

In [ ]:
import os

base = "/content/drive/MyDrive/BearingProject/CWRU"
for folder in os.listdir(base):
    full = os.path.join(base, folder)
    if os.path.isdir(full):
        print("----", folder, "----")
        print(os.listdir(full))
